# Supervised Classification Module (exercise)

**Lecturer:** Ashish Mahabal<br>
**Jupyter Notebook Authors:** Ashish Mahabal & Yuhan Yao

This is a Jupyter notebook lesson extending the GROWTH Summer School 2019 and adapted for the NARIT-EACOA 2019 summer workshop.

## Objective
Classify different classes using (a) decision trees and (b) random forest 

## Key steps
- Pick variable types
- Select training sample
- Select method
- Look at confusion matrix and details 

## Required dependencies

See GROWTH school webpage for detailed instructions on how to install these modules and packages.  Nominally, you should be able to install the python modules with `pip install <module>`.  The external astromatic packages are easiest installed using package managers (e.g., `rpm`, `apt-get`).

### Python modules
* python 3
* astropy
* numpy
* astroquery
* pandas
* matplotlib
* pydotplus
* IPython.display
* sklearn

### External packages
None

### Partial Credits
Pavlos Protopapas (LSSDS notebook)

## Now you will do a similar set of analyses on a different dataset.
### Here you will use the light curves file to derive features
### And then use the resulting file to run decision trees and random forest on that for classification

#### import the required modules (exercise)

In [1]:
import numpy as np
import pandas as pd
import os

#### read the lightcurves file

In [2]:
datadir = 'data'
lightcurves = datadir + '/CRTS_6varclasses.csv.gz'

In [3]:
lcs = pd.read_csv(lightcurves,
                 compression='gzip',
                 header=1,
                 sep=',',
                 skipinitialspace=True,
                 nrows=100000)
                 #skiprows=[4,5])
                 #,nrows=100000)

lcs.columns = ['ID', 'MJD', 'Mag', 'magerr', 'RA', 'Dec']
lcs.head()

,ID,MJD,Mag,magerr,RA,Dec
0,1109065026725,53705.501925,16.943797,0.082004,182.25871,9.76580
1,1109065026725,53731.483314,16.645102,0.075203,182.25867,9.76585
2,1109065026725,53731.491406,16.693791,0.076497,182.25870,9.76574
3,1109065026725,53731.499465,16.793651,0.078755,182.25869,9.76576
4,1109065026725,53731.507529,16.767817,0.077436,182.25878,9.76581


In [4]:
len(lcs)

100000

#### We need classes, so load the catalog file too

In [5]:
catalog = datadir + '/CatalinaVars.tbl.gz'

In [6]:
cat = pd.read_csv(catalog,
                 compression='gzip',
                 header=5,
                 sep=' ',
                 skipinitialspace=True,
                 )

columns = cat.columns[1:]
cat = cat[cat.columns[:-1]]
cat.columns = columns

cat.head()

,Catalina_Surveys_ID,Numerical_ID,RA_J2000,Dec,V_mag,Period_days,Amplitude,Number_Obs,Var_Type
0,CSS_J000020.4+103118,1109001041232,00:00:20.41,+10:31:18.9,14.62,1.491758,2.39,223,2
1,CSS_J000031.5-084652,1009001044997,00:00:31.50,-08:46:52.3,14.14,0.404185,0.12,163,1
2,CSS_J000036.9+412805,1140001063366,00:00:36.94,+41:28:05.7,17.39,0.274627,0.73,158,1
3,CSS_J000037.5+390308,1138001069849,00:00:37.55,+39:03:08.1,17.74,0.30691,0.23,219,1
4,CSS_J000103.3+105724,1109001050739,00:01:03.37,+10:57:24.4,15.25,1.5837582,0.11,223,8


In [7]:
RRd = cat[ cat['Var_Type'].isin([6]) & (cat['Number_Obs']>100) ]

In [8]:
RRd.head()

,Catalina_Surveys_ID,Numerical_ID,RA_J2000,Dec,V_mag,Period_days,Amplitude,Number_Obs,Var_Type
115,CSS_J001420.8+031214,1104002007409,00:14:20.84,+03:12:14.0,17.45,0.3871100,0.56,174,6
198,CSS_J001724.9+200542,1121002007726,00:17:24.90,+20:05:42.2,16.64,0.3571291,0.39,224,6
214,CSS_J001812.9+210201,1121002027610,00:18:12.97,+21:02:01.5,14.54,0.41616,0.34,224,6
531,CSS_J003001.7+094947,1109003028079,00:30:01.71,+09:49:47.6,16.91,0.3729404,0.36,212,6
640,CSS_J003359.4+022609,1101004049971,00:33:59.48,+02:26:09.0,15.87,0.3601025,0.27,195,6


### Get numerical ids of objects belonging to the RRd class - call them RRds

In [9]:
RRds = RRd['Numerical_ID']

In [10]:
RRds.head()

115    1104002007409
198    1121002007726
214    1121002027610
531    1109003028079
640    1101004049971
Name: Numerical_ID, dtype: int64

### Let us extract some features from the mags (lets ignore the mag errors for now)

#### For a given id you could do it as follows. isin() accepts a list so you could use the entire RRds there
#### you will lose the id infor if you do that in a single step, so you could break it up

In [11]:
lcs[lcs['ID'].isin(['1109065026725'])]['Mag']

0      16.943797
1      16.645102
2      16.693791
3      16.793651
4      16.767817
         ...    
394    16.461535
395    16.891826
396    16.824519
397    16.787664
398    16.816125
Name: Mag, Length: 399, dtype: float64

In [12]:
lcs[lcs['ID'].isin(RRds)]['Mag']

1212     14.998325
1213     14.984769
1214     15.010683
1215     14.984963
1216     14.817003
           ...    
98439    16.241040
98440    16.298755
98441    16.282934
98442    16.270512
98443    16.301170
Name: Mag, Length: 6340, dtype: float64

#### Lets assign mags for '1109065026725' to mags (a dictionary)

In [13]:
mags = {}
mags['1109065026725'] = lcs[lcs['ID'].isin(['1109065026725'])]['Mag']

#### Let us get the mean of mags for this one particular object: '1109065026725'

In [14]:
np.mean(mags['1109065026725'].values)

16.717012834586466

#### Assign it to another dictionary with the same key

In [15]:
means = {}
means['1109065026725'] = np.mean(mags['1109065026725'].values)

## Exercise!

### Get mean, median, skew, kurtosis for all ids in our light curves set

In [16]:
from scipy.stats import skew, kurtosis

print("id: 1109065026725")
print("mean: ", np.mean(mags['1109065026725']))
print("median: ", np.median(mags['1109065026725']))
print("skew: ", skew(mags['1109065026725']))
print("kurtosis: ", kurtosis(mags['1109065026725']))

id: 1109065026725
mean:  16.717012834586466
median:  16.729198
skew:  -0.19222112044199696
kurtosis:  -1.2417932925339292


In [17]:
lcs[lcs['ID'].isin(['1109065026725'])]['Mag']

0      16.943797
1      16.645102
2      16.693791
3      16.793651
4      16.767817
         ...    
394    16.461535
395    16.891826
396    16.824519
397    16.787664
398    16.816125
Name: Mag, Length: 399, dtype: float64

#### Define the dictionaries that you need

In [18]:
mags = {}
mean_dict = {}
median_dict = {}
skew_dict = {}
kurtosis_dict = {}

In [19]:
for idx in lcs['ID']:
    mag = lcs[lcs['ID'].isin([idx])]['Mag']
    mags[idx] = mag
    mean_dict[idx] = np.mean(mag)
    median_dict[idx] = np.median(mag)
    skew_dict[idx] = skew(mag)
    kurtosis_dict[idx] = kurtosis(mag)

## Now run decision tree and random forest using these variables by picking a couple of classes